In [1]:
import numpy as np
import random
from matplotlib import pyplot as plt
from tkinter import *
from tkinter import ttk
from turtle import *

In [2]:
# idée pour la gestion des temps d'infection /vaccin , création d'un deuxieme vecteur 2d comportant les durées de vie des états actuels

In [2]:
# paramètres
#taille monde:
n = 5
m = 7
#taux d'infection :
rzero = 2

In [51]:
#génère un monde de taille n,m

def generate_spatial_world(n,m):
    return np.random.choice([0,None],size=(n,m),p = [0.9,0.1])
worldtest = generate_spatial_world(n,m)


In [50]:
#génère la premiere infection aléatoire d'un monde de taille n,m

def infection_world(world):
    while True:
        n = random.randint(0,len(world[:,0])-1)
        m = random.randint(0,len(world[0,:])-1)
        if world[n,m] == 0:
            world[n,m] = 1
            break
    return world

In [48]:
#infecte la cellule x,y, si elle n'est pas vaccinée

def infection_cellule(world,x,y):
    worldc = world
    if world[x,y] != 2:
        worldc[x,y] = 1
    return world

In [47]:
#génère un tableau en 1d du voisinage de la case nm

def voisins(world,n,m):
    voisins=[]
    for i in range(0,len(world[:,0])):
        for j in range(0,len(world[0,:])):
            if max(abs(i-n),abs(j-m))<=1 and not(i==n and j == m) :
                voisins.append(world[i,j])
    return voisins

In [39]:
#vérifie si il y autre chose que des malades et cases vides dans le voisinage

def verif_voisins(world,n,m):
    for i in voisins(world,n,m):
        if i != None and i!= 1:
            return True
    return False

In [46]:
#fais évoluer le monde en effectuant un tour( pour le moment seul les nouvels infections sont gérées)

def tour(world,rzero):
    worldbis = world
    for i in range (0, len(world[:,0])):
        for j in range ( 0, len(world[0,:])):
            if world[i,j] == 1 and verif_voisins(worldbis,i,j):
                nb = 0 
                while nb < rzero and verif_voisins(worldbis,i,j):
                    x = i + (random.randint(-1,1))
                    y = j + (random.randint(-1,1))
                    if (x >= 0 and x <= len(world[:,0])-1) and (y>= 0 and y<= len(world[0,:])-1):
                        if worldbis[x,y] != None and worldbis[x,y]!= 1:
                            worldbis = infection_cellule(worldbis,x,y)
                            nb += 1
    return worldbis


In [41]:
#effectue le deplacement d'un membre du monde choisi de manière aléatoire

def deplacement(world):
    worldbis = world
    x1 = random.randint(0,len(world[:,0])-1)
    y1 = random.randint(0,len(world[0,:])-1)
    x2 = random.randint(0,len(world[:,0])-1)
    y2 = random.randint(0,len(world[0,:])-1)  
    while world[x1,y1] == None or (x1 == x2 and y1 == y2):
        x1 = random.randint(0,len(world[:,0])-1)
        y1 = random.randint(0,len(world[0,:])-1) 
    worldbis[x2,y2] = world[x1,y1]
    worldbis[x1,y1] = world[x2,y2]
    return world

In [42]:
#effectue x tour d'évolution du monde en ajoutant un déplacement aléatoire tous les freq_depla tours 

def tour_boucle(world,rzero,nbtour,freq_depla):
    i = 0
    while i < nbtour:
        if i%freq_depla == 0:
            world = deplacement(world)
        world = tour(world,rzero)
        i+= 1
    return world

In [45]:
def cycle_entier(n,m,nbtour,freq_depla,rzero):
    world = generate_spatial_world(n,m)
    world = infection_world(world)
    worldeux = world
    world = tour_boucle(world,rzero,nbtour,freq_depla)
    worldeux = tour_boucle(worldeux,rzero,nbtour,freq_depla)
    return world , worldeux
    
cycle_entier(n,m,10,5,2)

def print_cycle_entier(n,m,nbtour,freq_depla,rzero):
    print( cycle_entier(n,m,nbtour,freq_depla,rzero))


In [1]:
#essaie tkinter
def fenetre():
    fenetre = Tk()
    fenetre.geometry('800x800')
    fenetre.title('Sars-CoV2')
    fenetre['bg'] = 'black'
    start = Button(fenetre, text ="Simuler" , command =lambda: print_cycle_entier(n,m,10,5,2))
    start.pack()
    fenetre.mainloop()
fenetre()

NameError: name 'Tk' is not defined